In [4]:
import torch
import os

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from ultralytics import YOLO

In [ ]:
IMAGE_SIZE = 224
BATCH_SIZE = 16
NUM_WORKERS = 2

TRAINED_MODEL_PATH = 'best.pt'

In [6]:
model = YOLO(model=TRAINED_MODEL_PATH)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

In [7]:
all_preds = []
all_labels = []

project_path = './runs/classify'
test_dataset_path = '../../../../dataset/test'

class_names = os.listdir(test_dataset_path)
class_names = sorted(class_names)

for class_name in class_names:
    folder_path = os.path.join(test_dataset_path, class_name)

    results = model.predict(
        source=folder_path,
        imgsz=IMAGE_SIZE,
        project=project_path,
        device=device,
        batch=BATCH_SIZE,
        workers=NUM_WORKERS,
        save=True,
        save_txt=False,
        exist_ok=True,
    )

    for r in results:
        prediction_idx = torch.argmax(r.probs.data).item()
        prediction = model.names[prediction_idx]
        all_preds.append(prediction)
        all_labels.append(class_name)


image 1/170 /home/oz31/code/personal/python/training-skripsi/training-cnn/yolo/30_epochs/v8/../../../../dataset/test/algal_spot/003_0306.jpg: 224x224 algal_spot 1.00, red-rust 0.00, white-spot 0.00, gray_blight 0.00, helopeltis 0.00, 1.1ms
image 2/170 /home/oz31/code/personal/python/training-skripsi/training-cnn/yolo/30_epochs/v8/../../../../dataset/test/algal_spot/003_0307.jpg: 224x224 algal_spot 1.00, white-spot 0.00, red-rust 0.00, gray_blight 0.00, helopeltis 0.00, 1.1ms
image 3/170 /home/oz31/code/personal/python/training-skripsi/training-cnn/yolo/30_epochs/v8/../../../../dataset/test/algal_spot/003_0389.jpg: 224x224 algal_spot 0.98, red-rust 0.02, white-spot 0.00, red-spider-infested 0.00, red_spot 0.00, 1.1ms
image 4/170 /home/oz31/code/personal/python/training-skripsi/training-cnn/yolo/30_epochs/v8/../../../../dataset/test/algal_spot/003_0395.jpg: 224x224 red-rust 0.98, algal_spot 0.02, red_spot 0.00, white-spot 0.00, gray_blight 0.00, 1.1ms
image 5/170 /home/oz31/code/persona

In [8]:
cm = confusion_matrix(all_labels, all_preds)
print("Confusion Matrix:\n", cm)

report = classification_report(all_labels, all_preds, target_names=class_names, digits=2, zero_division=0)
print("Classification Report:\n", report)

correct = sum(p == t for p, t in zip(all_preds, all_labels))
total = len(all_labels)
overall_accuracy = (correct/total) * 100 if total > 0 else 0.0
print("Overall Accuracy: ", overall_accuracy)

Confusion Matrix:
 [[168   0   0   0   0   2   0   0   0]
 [  0 131   3   0   0   0   0   0   0]
 [  3   3 156   0   0   0   0   1   0]
 [  0   0   0 149   0   0   0   1   0]
 [  1   0   0   3 146   0   0   0   0]
 [  1   0   1   0   0  22   0   0   0]
 [  0   0   0   0   0   0  21   0   0]
 [  0   0   0   1   0   0   0 171   0]
 [  0   0   0   0   0   0   0   0  11]]
Classification Report:
                      precision    recall  f1-score   support

         algal_spot       0.97      0.99      0.98       170
       brown_blight       0.98      0.98      0.98       134
        gray_blight       0.97      0.96      0.97       163
            healthy       0.97      0.99      0.98       150
         helopeltis       1.00      0.97      0.99       150
           red-rust       0.92      0.92      0.92        24
red-spider-infested       1.00      1.00      1.00        21
           red_spot       0.99      0.99      0.99       172
         white-spot       1.00      1.00      1.00     